# 1. 기성보고현황

# 1. 공사정보 목록에서 "SPTNO" 추출

In [1]:
from common import commonFunc as cf
import pandas as pd
import pickle

# pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")


# 기존: APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"
### APIKEY 불러오기 ###
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)
SITENAME = "건설사업정보시스템"
DATANAME= "공사정보 목록"

targetData = metadata.loc[metadata.자료명==DATANAME]



SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
# preSetFolder = targetData["저장폴더"].values[0]

# preSetFolder

custrunctionListDt = cf.loaddata(SITENAME,DATANAME,SERVICENAME)

targetColumn = "sptno"

sptNoList = custrunctionListDt[targetColumn].drop_duplicates().tolist()

# 2. 연도별 공사계약 목록에서 "SPTTO" 추출 * SPTNO 활용

In [2]:
SITENAME = "건설사업정보시스템"
DATANAME= "연도별 공사계약 목록"

In [3]:
targetData = metadata.loc[metadata.자료명==DATANAME]

In [4]:
# ##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]

In [5]:
preData = cf.loaddata(SITENAME,DATANAME,SERVICENAME)

In [6]:
dfCombi = preData.loc[:,["sptno","sptto"]].drop_duplicates()

In [7]:
# SITENAME = "건설사업정보시스템"
# DATANAME= "연도별 공사계약 목록"
# # 기존: APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"
# ### APIKEY 불러오기 ###
# with open("../input/calsapikey.pickle","rb") as fr:
#     APIKEY = pickle.load(fr)

# targetData = metadata.loc[metadata.자료명==DATANAME]

# ##### 파라미터 설정 #####
# URL = targetData["URL"].values[0]
# SERVICENAME = targetData["서비스키"].values[0]
# SERVICENAME = SERVICENAME.split(".")[0]
# REQPARAM = targetData["요청변수"].values[0]
# REQPARAM = REQPARAM.split(",")
# PRIMARYKEY = targetData["기본키"].values[0]

In [8]:
# #### 페이지번호 설정 ###
# PAGEYN=1
# if REQPARAM.count("pageyn") == 0:
#     PAGEYN = 0
# else:
#     PAGEYN = 1

# ### 개별 코드 작업 영역 ###

# sptNoLen = len(sptNoList)

# resultDfMerged = pd.DataFrame()

# for i in range(0, sptNoLen):
#     BASEPARAM={"serviceKey":APIKEY, "type":"json", "sptNo": sptNoList[i]}
#     resultDf = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN)
#     resultDfMerged = resultDfMerged.append(resultDf)

In [9]:
# cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

# 3. 기성보고 현황 수집

In [10]:
SITENAME = "건설사업정보시스템"
DATANAME= "기성보고 현황"

targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [11]:
JSONKEY = "detail1"
DUMMY = 0

In [12]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageNo") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1

In [13]:
newParam = dfCombi.copy()

In [14]:
orgParam = []
try:
    # 기존 데이터 및 파라미터 정보 불러오기
    # 리스트로 불러오기로 변경 0425
    orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)[0]
    outData = cf.loaddata(SITENAME,DATANAME,SERVICENAME)
## 파일이 없는경우
except Exception as e:
    orgParam = []
    print(e)

ptlcmnoList = list( set(newParam) - set(orgParam) )
# 모드: 0=종료 1=append 2=새로생성
mode = 2

# 업데이트 할 내용이 없으면 종료
if ptlcmnoList == []:
    mode = 0
    print("{} 정보 quit모드 {} ".format(DATANAME, mode))
    quit()
# orgParam == []
elif orgParam == []:
    mode = 2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))
# 이외에는 append 모드
else:
    mode = 1
    print("{} 정보 append 모드 {} ".format(DATANAME, mode))

[Errno 2] No such file or directory: '../output\\건설사업정보시스템\\기성보고 현황\\selectAcpsRprtList.pickle'
기성보고 현황 정보 new 모드 2 


In [15]:
ptlcmnoListLen = len(newParam)
resultDfMerged = pd.DataFrame()
pageList = [] 

# ptlcmnoListLen = len(ptlcmnoList)
APICALL = 0

In [ ]:
for i  in range(0,ptlcmnoListLen):
    sptnoValue = newParam.loc[i].sptno
    spttoValue = newParam.loc[i].sptto
    BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "sptTo": spttoValue, "type":"json"}
    # resultDf = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
    # scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN)
    scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN,APIKEY,APICALL, jsonkey=JSONKEY, dummy=DUMMY) #추가
    resultDf = scrapyResult[0]
    pageList.append( scrapyResult[1] )
    APICALL = scrapyResult[2]  #추가
    resultDfMerged = resultDfMerged.append(resultDf)
    print("{}param finished".format(i))

1 page scraping start apicall iter: 1 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
dataframe기성보고 현황, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C1999006', 'sptTo': 9, 'type': 'json'} rows: 24 completed
0param finished
1 page scraping start apicall iter: 2 / used 9D486CED-81E4-4884-96FF-BB1F14A9223B
0 no pageNo
dataframe기성보고 현황, param:{'serviceKey': '9D486CED-81E4-4884-96FF-BB1F14A9223B', 'sptNo': 'C1999006', 'sptTo': 8, 'type': 'json'} rows: 24 completed
1param finished
1 page scraping start apicall iter: 3 / used 0974320D-C689-4543-8464-10DE1E5505A9
0 no pageNo
dataframe기성보고 현황, param:{'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C1999006', 'sptTo': 7, 'type': 'json'} rows: 24 completed
2param finished
1 page scraping start apicall iter: 4 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
dataframe기성보고 현황, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C1999006', 'sptTo': 6, 'type': 'json'} rows: 2

In [ ]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME, mode)

resultDfMerged.shape

In [ ]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
ptlcmnoList = orgParam + ptlcmnoList

paramList = [ptlcmnoList,pageList]
# 최종 파라미터 저장
cf.saveparam(paramList, SITENAME,DATANAME,SERVICENAME)

In [ ]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

기성보고 현황 save compled


In [17]:
resultDfMerged.shape

(7278, 24)